In [22]:
!pip install tweepy
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 MB 5.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 8.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12


In [1]:
import tweepy
import os.path as path
import numpy as np
import nltk.tokenize as tk
import math
import scipy.stats as stats
import pandas as pd
import huggingface_hub
from scipy.stats import pearsonr
from transformers import AutoTokenizer
from transformers import AutoModelForMaskedLM
import torch
from transformers import BertModel
import sklearn.cluster
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import random

In [2]:
def get_all_examples():
    """Load all examples from a CSV file using pandas and return those that contain text.

    Returns:
        A list of tweets (or data) that contain text.
    """
    save_file = './twitter_emoji.csv'
    
    if path.exists(save_file):
        df = pd.read_csv(save_file, lineterminator = '\n')
        ids_to_examples = df[df['Text'].notna()].to_dict(orient='records')  # Convert to list of dicts
    else:
        print('Could not find tweets, so returning an empty list!')
        ids_to_examples = []

    return ids_to_examples


def get_emojis_in_tweet(tweet, emojis, tokenizer):
    """Get a list of all the emojis in a tweet based on the sets provided

    Args:
        tweet: Tweet
        tokenizer: NLTK tokenizer

    Returns:
        Emojis in a tweet
    """
    tokens = tokenizer.tokenize(tweet['Text'])
    ems = set()
    for token in tokens:
        if token in emojis:
            ems.add(token)
    return ems


def get_tweets_with_emoji(tweets, emojis):
    """Get all tweets with emoji in the sets

    Args:
        tweets: List of Tweets

    Returns:
        All tweets containing emoji

    """
    tokenizer = tk.TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
    ems = list()
    for tweet in tweets:
        if get_emojis_in_tweet(tweet, emojis, tokenizer):
            ems.append(tweet)
    return ems


In [3]:
tweets = get_all_examples()
tweetsWithPointRight = get_tweets_with_emoji(tweets, ["👉"])
lines = []
for tweet in tweetsWithPointRight:
    lines.append(tweet["Text"])

lines

['🧡@KeplerHomes AirdropBox event for #Arbitrum ecological users is here. A total of 550,000 addresses are eligible for #airdrop, and 5 types of AirDropbox with different scarcity can be issued.\n\n💙Invitation code: 52DC39\n🏆Airdrop Portal:👉 https://t.co/fudohu97uV',
 'Remember, success in online business is a marathon, not a sprint. Keep at it, stay focused, and success will come." #patience #onlinebusiness #success\nFor more tips and Strategies, follow me 👉 @coach_lawrence1 https://t.co/IvtL9Om86J',
 '@occupied_9 @Rhiannon_clare_ @FightHaven Thanks for the update the sh*t country the sh*t police 👉💩😡',
 "📢 It's confirmed. Whitelist for Shardeum Airdrop is here.\n\nYou can now join our whitelist, there are over 30000 spots. (First come First Served)\n👉 https://t.co/aHulc12YTO\n\nThe amount of tokens you receive during the Airdrop will depend on your portfolio and testnet status. https://t.co/DiukBFV93K",
 '7/ What to do next?\n\n-Using these bridges:\n👉https://t.co/NwvBsrpz7F\n(Polygon-

In [6]:
checkpoint = 'bert-base-uncased'

model = AutoModelForMaskedLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.add_tokens(["👉"])

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
# and adapting the other function
def combined_vector_for(targetword, text, bert_input, bert_output,
                        layer = -1, word_occurrence = 0):
    """
    calculate a word vector
    based on the mean of the WordPiece vectors in the given layer.
    targetword is a word appearing in text.
    bert_input is the whole dictionary returned by the tokenizer.
    bert_output is the last-layer output obtained
    on this text.
    """
    tokenized_text = [w for w, n in tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)]

    # where in the tokenized text (split into words but not split into word pieces)
    # do we find the target word?
    target_word_indices = [i for i, x in enumerate(tokenized_text) if x == targetword]

    # sanity check
    if len(target_word_indices) < word_occurrence + 1:
        # no occurrences found, or not enough to match the required one
        return None

    # if multiple occurrences of the target word, we use this one
    usethis_targetword_index = target_word_indices[ word_occurrence]

    # use word_to_tokens to determine the word piece span of the target word
    word_start, word_end = bert_input.word_to_tokens( usethis_targetword_index )

    # extract the embeddings from the right layer, and the target vectors from that layer
    embeddings= bert_output["hidden_states"][layer]
    # print(embeddings.shape)
    target_vectors = embeddings[0, word_start:word_end, :]

    # if we have multiple word pieces for this word, average over them
    avg_target_vector = target_vectors.mean(dim = 0)

    return avg_target_vector.detach().numpy()



In [ ]:
embeddings = []
# checkpoint = 'bert-base-uncased'
for line in lines:

    text = line

    bert_inputs = tokenizer(text, padding=True, truncation=True,  return_tensors="pt")

    bert_output = model(bert_inputs["input_ids"], output_hidden_states = True)

    charge_output = combined_vector_for("👉", text, bert_inputs, bert_output, layer = 7, word_occurrence = 0)
    embeddings.append(charge_output)

print(len(embeddings))
# print(embeddings)

In [ ]:
kmeans = sklearn.cluster.KMeans(n_clusters=5, random_state=2048).fit(embeddings)
#random_state makes clustering deterministic - it's a seed value

In [ ]:
cluster_list = kmeans.predict(embeddings)
print(list(cluster_list))

In [ ]:
cluster_dict = {0:[],1:[],2:[],3:[],4:[]}
for i in range(len(cluster_list)):
  cluster_dict[cluster_list[i]].append(i)

In [ ]:
outs = 3
random.seed(50)

for cluster in cluster_dict.keys():
  sents = random.sample(cluster_dict[cluster],outs)
  for sent in sents:
    print(lines_list[sent])

  print()

In [ ]:
twodim = PCA().fit_transform(embeddings)[:,:2]

# set up the canvas
fig, ax =  plt.subplots()

# add a scatter plot of the two-D embeddings
scatter = ax.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c=cluster_list)

# and show the canvas
plt.show()